In [ ]:
STYLE_ID = 14  # スタイルID
VVM_ID = 1  # VVMファイル名
# VVMファイル名はスタイルIDに対応する数値を設定してください。一覧はこちら
# https://github.com/VOICEVOX/voicevox_vvm/blob/main/README.md#%E9%9F%B3%E5%A3%B0%E3%83%A2%E3%83%87%E3%83%ABvvm%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%A8%E5%A3%B0%E3%82%AD%E3%83%A3%E3%83%A9%E3%82%AF%E3%82%BF%E3%83%BC%E3%82%B9%E3%82%BF%E3%82%A4%E3%83%AB%E5%90%8D%E3%81%A8%E3%82%B9%E3%82%BF%E3%82%A4%E3%83%AB-id-%E3%81%AE%E5%AF%BE%E5%BF%9C%E8%A1%A8

FILE_DIRECTORY = (
    "/content/drive/MyDrive/textfiles/"  # 処理したいテキストファイルがあるディレクトリ
)

TEMPO = 1.0  # 再生速度の調整

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title soxのインストール

!sudo apt install -y sox libsox-fmt-mp3

In [ ]:
# @title Voicevoxのインストール

%%bash
curl -sSfL https://github.com/VOICEVOX/voicevox_core/releases/latest/download/download-linux-x64 -o download
chmod u+x download
yes | ./download --exclude c-api --devices cuda

In [ ]:
!pip install https://github.com/VOICEVOX/voicevox_core/releases/download/0.16.0/voicevox_core-0.16.0-cp310-abi3-manylinux_2_34_x86_64.whl

In [ ]:
# @title ユーザー辞書の設定
# 読み間違えやすい単語などを登録できる

from voicevox_core import UserDictWord
from voicevox_core.blocking import UserDict

user_dict = UserDict()
user_dict.add_word(UserDictWord("苫米地", "トマベチ", 1, priority=6))

# ユーザー辞書を保存したい場合
# user_dict.save("./user_dict.json")

# 保存したユーザー辞書をロードしたい場合
# user_dict.load("./user_dict.json")

In [ ]:
from voicevox_core.blocking import Onnxruntime, OpenJtalk, Synthesizer, VoiceModelFile

# 1. Synthesizerの初期化
voicevox_onnxruntime_path = (
    "/content/voicevox_core/onnxruntime/lib/" + Onnxruntime.LIB_VERSIONED_FILENAME
)
open_jtalk_dict_dir = "/content/voicevox_core/dict/open_jtalk_dic_utf_8-1.11"
synthesizer = Synthesizer(
    Onnxruntime.load_once(filename=voicevox_onnxruntime_path),
    OpenJtalk(open_jtalk_dict_dir),
)

# 2. 音声モデルの読み込み
with VoiceModelFile.open(f"/content/voicevox_core/models/vvms/{VVM_ID}.vvm") as model:
    synthesizer.load_voice_model(model)

# ユーザー辞書の読み込み
synthesizer.open_jtalk.use_user_dict(user_dict)


def create_audio(text, filename):
    print(filename)

    # 3. テキスト音声合成
    style_id = STYLE_ID
    wav = synthesizer.tts(text, style_id)
    with open("temp.wav", "wb") as f:
        f.write(wav)
    !sox temp.wav {filename} pad 0 0.5

In [ ]:
def merge_audio(filename):
    %cd $FILE_DIRECTORY

    # 中間ファイルの結合、ノーマライズ
    !sox {filename}_*.wav {filename}_merged.mp3 norm -0.1 tempo {TEMPO}

    # 中間ファイル削除
    %rm {filename}_*.wav
    %rm temp.wav

In [ ]:
import os
import glob

file_list = sorted(glob.glob(os.path.join(FILE_DIRECTORY, "*.txt")))
for file in file_list:
    with open(file, "r", encoding="utf-8") as f:
        print(file)
        for i, line in enumerate(f):
            text = line
            output_file = os.path.join(
                FILE_DIRECTORY,
                f"{os.path.splitext(os.path.basename(file))[0]}_{i:05d}.wav",
            )
            if os.path.exists(output_file):
                continue
            else:
                try:
                    create_audio(text, output_file)
                except Exception as e:
                    print("create_audio exception")
    merge_audio(os.path.splitext(os.path.basename(file))[0])  # マージする